### Passo 1 - Importar Arquivos e Bibliotecas

In [2]:
import pandas as pd
import win32com.client as win32
import pathlib
import os
import yagmail

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [3]:
emails = pd.read_excel(r'Bases de dados\Emails.xlsx')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', sep=';', encoding='latin1')


### Passo 3 - Salvar a planilha na pasta de backup

In [4]:
#incluir loja no vendas

vendas = vendas.merge(lojas, on= 'ID Loja')

In [5]:
dicionario_lojas = {}

for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja'] == loja,:]

In [6]:
dia_indicador = vendas['Data'].max()

In [7]:
caminho_backup = pathlib.Path(r"Backup Arquivos Lojas")

arquivos_pasta_backup = caminho_backup.iterdir() ##verifica itens dentro da pasta
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
        
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month,dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Calcular o indicador para 1 loja

In [8]:
loja = 'Norte Shopping'
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas.loc[vendas['Data'] == dia_indicador, :]

#faturamento
faturamento_ano = vendas_loja['Valor Final'].sum()
print(faturamento_ano)
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(faturamento_dia)

#diversidade produtoos

qtde_produtos_ano = len(vendas_loja['Produto'].unique())
print(qtde_produtos_ano)

qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
print(qtde_produtos_dia)


#ticket medio an

valor_venda = vendas_loja.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
print(ticket_medio_ano)


valor_venda = vendas_loja_dia.groupby('Código Venda').sum()
ticket_medio_dia = vendas_loja_dia['Valor Final'].mean()
print(ticket_medio_dia)

1711968
21591
120
35
784.5866177818515
553.6153846153846


C:\Users\franc\AppData\Local\Temp\ipykernel_12356\1958962488.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  valor_venda = vendas_loja.groupby('Código Venda').sum()
C:\Users\franc\AppData\Local\Temp\ipykernel_12356\1958962488.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  valor_venda = vendas_loja_dia.groupby('Código Venda').sum()


### Passo 5 - Enviar por e-mail para o gerente

In [9]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano =120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500 

In [10]:
import yagmail

usuario = yagmail.SMTP(user='franciscojdsjr@gmail.com', password='rywonlhxsedezoil')
nome = emails.loc[emails['Loja'] == loja, 'Gerente'].values[0]
destinatario = emails.loc[emails['Loja'] == loja, 'E-mail'].values[0]
copia = ['francisco-junior@outlook.com']

if faturamento_dia >= meta_faturamento_dia:
    cor_fat_dia = 'green'
else:
    cor_fat_dia = 'red'
if faturamento_ano >= meta_faturamento_ano:
    cor_fat_ano = 'green'
else:
    cor_fat_ano = 'red'
if qtde_produtos_dia >= meta_qtdeprodutos_dia:
    cor_qtde_dia = 'green'
else:
    cor_qtde_dia = 'red'
if qtde_produtos_ano >= meta_qtdeprodutos_ano:
    cor_qtde_ano = 'green'
else:
    cor_qtde_ano = 'red'
if ticket_medio_dia >= meta_ticketmedio_dia:
    cor_ticket_dia = 'green'
else:
    cor_ticket_dia = 'red'
if ticket_medio_ano >= meta_ticketmedio_ano:
    cor_ticket_ano = 'green'
else:
    cor_ticket_ano = 'red'

corpoemail  = f'''
<p> Bom dia! {nome} </p>

<p> o resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi: </p>

<table>
  <tr>
    <th>Indicador</th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style="text-align: center">R${faturamento_dia:.2f}</td>
    <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
    <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style="text-align: center">{qtde_produtos_dia}</td>
    <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
    <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
    <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
    <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
  </tr>
</table>
<table>
  <tr>
    <th>Indicador</th>
    <th>Valor Ano</th>
    <th>Meta Ano</th>
    <th>Cenário Ano</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style="text-align: center">R${faturamento_ano:.2f}</td>
    <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
    <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style="text-align: center">{qtde_produtos_ano}</td>
    <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
    <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
    <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
    <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
  </tr>
</table>

<p> segue em anexo a planilha com todos os dados para mais detalhes </p>

<p> Att., Francisco </p>
'''



anexo = pathlib.Path.cwd()  / caminho_backup / loja / '{}_{}_{}.xlsx'.format(dia_indicador.month,dia_indicador.day,loja)



##quando usamos 3 aspas conseguimos define que essa string será grande 

usuario.send(to= destinatario, \
            cc=copia, \
            subject='OnePage Dia {}/{} - Loja {}'.format(dia_indicador.day,dia_indicador.month,loja), \
            contents= corpoemail, \
            attachments= str(anexo) )


NameError: name 'copia' is not defined

In [ ]:
for loja in dicionario_lojas:   
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas.loc[vendas['Data'] == dia_indicador, :]
    
    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    
    #diversidade produtoos
    
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    
    #ticket medio an
    
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    
    
    valor_venda = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = vendas_loja_dia['Valor Final'].mean()
    
    
    meta_faturamento_dia = 1000
    meta_faturamento_ano = 1650000
    meta_qtdeprodutos_dia = 4
    meta_qtdeprodutos_ano =120
    meta_ticketmedio_dia = 500
    meta_ticketmedio_ano = 500 
    
    usuario = yagmail.SMTP(user='franciscojdsjr@gmail.com', password='rywonlhxsedezoil')
    nome = emails.loc[emails['Loja'] == loja, 'Gerente'].values[0]
    destinatario = emails.loc[emails['Loja'] == loja, 'E-mail'].values[0]
    copia = ['francisco-junior@outlook.com']

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    corpoemail  = f'''
    <p> Bom dia! {nome} </p>

    <p> o resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi: </p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p> segue em anexo a planilha com todos os dados para mais detalhes </p>

    <p> Att., Francisco </p>
    '''



    anexo = pathlib.Path.cwd()  / caminho_backup / loja / '{}_{}_{}.xlsx'.format(dia_indicador.month,dia_indicador.day,loja)



    ##quando usamos 3 aspas conseguimos define que essa string será grande 

    usuario.send(to= destinatario, \
                cc=copia, \
                subject='OnePage Dia {}/{} - Loja {}'.format(dia_indicador.day,dia_indicador.month,loja), \
                contents= corpoemail, \
                attachments= str(anexo) )



### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [19]:
faturamento_lojas_ano = vendas.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas_ano.sort_values(by= 'Valor Final',ascending=False)
display(faturamento_lojas_ano)

vendas_dia = vendas.loc[vendas['Data'] == dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by= 'Valor Final',ascending=False)
display(faturamento_lojas_dia)


C:\Users\franc\AppData\Local\Temp\ipykernel_12356\3950689904.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  faturamento_lojas_ano = vendas.groupby('Loja')[['Loja','Valor Final']].sum()


,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


C:\Users\franc\AppData\Local\Temp\ipykernel_12356\3950689904.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja','Valor Final']].sum()


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 8 - Enviar e-mail para diretoria